<a href="https://colab.research.google.com/github/cmlins/recuperacaoDeInformacao/blob/master/wrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests as rq
import re

# Sites Livros

1.   https://www.amazon.com.br/
2.   https://www.submarino.com.br/
3.   https://www.saraiva.com.br/
4.   https://www.estantevirtual.com.br/
5.   https://www.americanas.com.br/
6.   https://www.ciadoslivros.com.br/
7.   https://www.traca.com.br/
8.   https://www.livrariacultura.com.br
9.   https://www.disal.com.br/
10.  https://www.extra.com.br/livros/

##AMAZON

### All books in one page

In [0]:
url_list = ['https://www.amazon.com.br/Livros/b?ie=UTF8&node=6740748011&ref_=nav_books_all',
            'https://www.amazon.com.br/b/ref=s9_acss_bw_cg_livros_3a1_w?node=7872689011&pf_rd_m=A3RN7G7QC5MWSZ&pf_rd_s=merchandised-search-13&pf_rd_r=QVZ9EYBNWVBAX5VRE6KH&pf_rd_t=101&pf_rd_p=ef6255a0-96c8-4931-bc85-1e4d682f1064&pf_rd_i=6740748011',
            'https://www.amazon.com.br/b/ref=s9_acss_bw_cg_livros_2a1_w?node=16495864011&pf_rd_m=A3RN7G7QC5MWSZ&pf_rd_s=merchandised-search-13&pf_rd_r=G9RQPBBW9G6ZQ5XX82M8&pf_rd_t=101&pf_rd_p=ef6255a0-96c8-4931-bc85-1e4d682f1064&pf_rd_i=6740748011',
            'https://www.amazon.com.br/b/ref=s9_acss_bw_cg_livros_2d1_w?node=11443295011&pf_rd_m=A3RN7G7QC5MWSZ&pf_rd_s=merchandised-search-13&pf_rd_r=0AC9WCXZN61JJGX49F4H&pf_rd_t=101&pf_rd_p=ef6255a0-96c8-4931-bc85-1e4d682f1064&pf_rd_i=6740748011',
            'https://www.amazon.com.br/b/ref=s9_acss_bw_cg_livros_4b1_w?node=17021500011&pf_rd_m=A3RN7G7QC5MWSZ&pf_rd_s=merchandised-search-13&pf_rd_r=0JX9SAGKQ4QCZ7DET92M&pf_rd_t=101&pf_rd_p=ef6255a0-96c8-4931-bc85-1e4d682f1064&pf_rd_i=6740748011',
            'https://www.amazon.com.br/b/ref=s9_acss_bw_cg_livros_2c1_w?node=7882627011&pf_rd_m=A3RN7G7QC5MWSZ&pf_rd_s=merchandised-search-13&pf_rd_r=068FWS2KD5K5K0RGNXRY&pf_rd_t=101&pf_rd_p=ef6255a0-96c8-4931-bc85-1e4d682f1064&pf_rd_i=6740748011',
            'https://www.amazon.com.br/b/ref=s9_acss_bw_cg_livros_2b1_w?node=16271670011&pf_rd_m=A3RN7G7QC5MWSZ&pf_rd_s=merchandised-search-13&pf_rd_r=068FWS2KD5K5K0RGNXRY&pf_rd_t=101&pf_rd_p=ef6255a0-96c8-4931-bc85-1e4d682f1064&pf_rd_i=6740748011',
            'https://www.amazon.com.br/b/ref=s9_acss_bw_cg_livros_3b1_w?node=7842710011&pf_rd_m=A3RN7G7QC5MWSZ&pf_rd_s=merchandised-search-13&pf_rd_r=068FWS2KD5K5K0RGNXRY&pf_rd_t=101&pf_rd_p=ef6255a0-96c8-4931-bc85-1e4d682f1064&pf_rd_i=6740748011',
            'https://www.amazon.com.br/b/ref=s9_acss_bw_cg_livros_3d1_w?node=7872552011&pf_rd_m=A3RN7G7QC5MWSZ&pf_rd_s=merchandised-search-13&pf_rd_r=068FWS2KD5K5K0RGNXRY&pf_rd_t=101&pf_rd_p=ef6255a0-96c8-4931-bc85-1e4d682f1064&pf_rd_i=6740748011',
            'https://www.amazon.com.br/b/ref=s9_acss_bw_cg_livros_4d1_w?node=5475882011&pf_rd_m=A3RN7G7QC5MWSZ&pf_rd_s=merchandised-search-13&pf_rd_r=068FWS2KD5K5K0RGNXRY&pf_rd_t=101&pf_rd_p=ef6255a0-96c8-4931-bc85-1e4d682f1064&pf_rd_i=6740748011']

In [0]:
def Amazon_col(site):
  page = rq.get(url)

  soup = bs(page.text,"html.parser")

  books = soup.find_all('li', {'class':'a-carousel-card acswidget-carousel__card'})

  books_list = []

  for book in books:
    book = re.sub("\n|\t|\xa0|[R][$]", " ", (book.text.strip()))
    book = re.sub("[,]", '.', book)
    books_list.append(book)
    
  for i in range(len(books_list)):
    books_list[i] = books_list[i].split('    ')
    books_list[i] = list(filter(None, books_list[i]))
    books_list[i] = [item.strip() for item in books_list[i]]
    if (len(books_list[i]) > 4):
      books_list[i] = [books_list[i][0],books_list[i][1],books_list[i][2],books_list[i][4]]

  book_list = []
  
  for book in books_list:
    if len(book) == 4:
      book_list.append(book)

  col = ['Título', 'Autor', 'Acabamento', 'Preço']

  books_pd = pd.DataFrame(book_list,\
                        columns=col)
  
  return books_pd

In [4]:
for url in url_list:
  print(url)
  display(Amazon_col(url)[:5])
  print('\n')

https://www.amazon.com.br/Livros/b?ie=UTF8&node=6740748011&ref_=nav_books_all


,Título,Autor,Acabamento,Preço
0,Do Mil ao Milhão. Sem Cortar o...,Thiago Nigro,Capa comum,34.90
1,O milagre da manhã,Hal Elrod. Marcelo Schild,Capa comum,39.90
2,A Sutil Arte de Ligar o F*da-Se: Uma...,Mark Manson. Joana Faro,Capa comum,34.90
3,Mais esperto que o diabo: O mistério...,Napoleon Hill,Capa comum,42.90
4,Os segredos da mente milionária,T. Harv Eker. Pedro Jorgensen Junior,Capa comum,34.90




https://www.amazon.com.br/b/ref=s9_acss_bw_cg_livros_3a1_w?node=7872689011&pf_rd_m=A3RN7G7QC5MWSZ&pf_rd_s=merchandised-search-13&pf_rd_r=QVZ9EYBNWVBAX5VRE6KH&pf_rd_t=101&pf_rd_p=ef6255a0-96c8-4931-bc85-1e4d682f1064&pf_rd_i=6740748011


,Título,Autor,Acabamento,Preço
0,Notas Autobiográficas,Albert Einstein,Capa dura,39.90
1,O Jovem Törless (Biblioteca Áurea),Robert Musil,Capa dura,39.90
2,Todos os Homens São Mortais,Simone De Beauvoir,Capa dura,59.90
3,Um Teto Todo Seu (Biblioteca Áurea),Virginia Woolf,Capa dura,39.90
4,A Casa Soturna (Biblioteca Áurea),Charles Dickens,Capa dura,79.90




https://www.amazon.com.br/b/ref=s9_acss_bw_cg_livros_2a1_w?node=16495864011&pf_rd_m=A3RN7G7QC5MWSZ&pf_rd_s=merchandised-search-13&pf_rd_r=G9RQPBBW9G6ZQ5XX82M8&pf_rd_t=101&pf_rd_p=ef6255a0-96c8-4931-bc85-1e4d682f1064&pf_rd_i=6740748011


...

,Título,Autor,Acabamento,Preço
0,O Eternauta 1969,Héctor Germán Oesterheld. alberto Breccia...,Capa dura,31.40
1,Akira - Volume 4,Katsuhiro Otomo,Capa comum,76.90
2,Battle Angel Alita - Vol. 4,Yukito Kishiro,Capa comum,44.90
3,Battle Angel Alita - Vol. 3,Yukito Kishiro,Capa comum,44.90
4,Battle Angel Alita - Vol. 1,Yukito Kishiro,Capa comum,44.90




https://www.amazon.com.br/b/ref=s9_acss_bw_cg_livros_3d1_w?node=7872552011&pf_rd_m=A3RN7G7QC5MWSZ&pf_rd_s=merchandised-search-13&pf_rd_r=068FWS2KD5K5K0RGNXRY&pf_rd_t=101&pf_rd_p=ef6255a0-96c8-4931-bc85-1e4d682f1064&pf_rd_i=6740748011


,Título,Autor,Acabamento,Preço
0,Confeitaria escalafobética...,Raiza Costa,Capa dura,98.00
1,A química dos bolos: Receitas e...,Joyce Galvão,Capa dura,129.90
2,Le Cordon Bleu : Todas as técnicas...,Jeni Wright. Eric Treuille,Capa dura,249.90
3,O Que Tem na Geladeira? 30 Legumes e...,Rita Lobo,Capa dura,86.00
4,Cozinha prática,Rita Lobo,Capa dura,90.00




https://www.amazon.com.br/b/ref=s9_acss_bw_cg_livros_4d1_w?node=5475882011&pf_rd_m=A3RN7G7QC5MWSZ&pf_rd_s=merchandised-search-13&pf_rd_r=068FWS2KD5K5K0RGNXRY&pf_rd_t=101&pf_rd_p=ef6255a0-96c8-4931-bc85-1e4d682f1064&pf_rd_i=6740748011


,Título,Autor,Acabamento,Preço
0,Laranja mecânica,Anthony Burgess,eBook Kindle,18.15
1,O Processo,Franz Kafka,eBook Kindle,2.54
2,As Aventuras de Tom Sawyer: Texto...,Mark Twain. Kerem Freitas...,eBook Kindle,17.91
3,O conde de Monte Cristo: edição bolso...,Alexandre Dumas,eBook Kindle,39.90
4,Jane Eyre (Edição Bilíngue),Charlotte Bronte. Fabio Cyrino...,eBook Kindle,8.91


### Information from a single book

In [0]:
# url = 'https://www.amazon.com.br/Livros/b?ie=UTF8&node=6740748011&ref_=nav_books_all'

# Amazon_col(url)[:10]

In [0]:
# 'https://www.amazon.com.br/Mil-Milh%C3%A3o-Sem-Cortar-Cafezinho/dp/8595083274/ref=sr_1_1?__mk_pt_BR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&keywords=Do+Mil+ao+Milh%C3%A3o.+Sem+Cortar&qid=1568583440&s=books&sr=1-1'
# 'https://www.amazon.com.br/milagre-manh%C3%A3-Hal-Elrod/dp/8576849941/ref=sr_1_1?__mk_pt_BR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&keywords=O+milagre+da+manh%C3%A3&qid=1568583491&s=gateway&sr=8-1'
# 'https://www.amazon.com.br/Sutil-Arte-Ligar-Se/dp/855100249X/ref=sr_1_1?__mk_pt_BR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=2RL0TN82EOIU0&keywords=a+sutil+arte+de+ligar+o+foda-se&qid=1568583959&s=books&sprefix=a+sutil%2Cstripbooks%2C281&sr=1-1'
# 'https://www.amazon.com.br/Esperto-Mist%C3%A9rio-Revelado-Liberdade-Sucesso/dp/8568014003/ref=sr_1_1?__mk_pt_BR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&keywords=Mais+esperto+que+o+diabo%3A+O+mist%C3%A9rio&qid=1568583579&s=books&sr=1-1'
# 'https://www.amazon.com.br/segredos-mente-milion%C3%A1ria-Harv-Eker/dp/8575422391/ref=sr_1_1?__mk_pt_BR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&keywords=Os+segredos+da+mente+milion%C3%A1ria&qid=1568583626&s=books&sr=1-1'
# 'https://www.amazon.com.br/Pai-rico-pai-pobre-anos/dp/8550801488/ref=sr_1_1?__mk_pt_BR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&keywords=Pai+rico.+pai+pobre+20+anos&qid=1568583668&s=books&sr=1-1'
# 'https://www.amazon.com.br/B%C3%ADblia-King-James-Atualizada/dp/8544105661/ref=sr_1_1?__mk_pt_BR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&keywords=B%C3%ADblia+King+James+Atualizada&qid=1568583720&s=books&sr=1-1'
# 'https://www.amazon.com.br/poder-a%C3%A7%C3%A3o-Paulo-Vieira/dp/854520034X/ref=sr_1_1?__mk_pt_BR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&keywords=O+poder+da+a%C3%A7%C3%A3o&qid=1568583773&s=books&sr=1-1'
# 'https://www.amazon.com.br/Como-Fazer-Amigos-Influenciar-Pessoas/dp/8504018024/ref=sr_1_1?__mk_pt_BR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&keywords=Como+Fazer+Amigos+e+Influenciar+Pessoas&qid=1568583810&s=books&sr=1-1'
# 'https://www.amazon.com.br/Me-Poupe-passos-faltar-dinheiro/dp/8543105811/ref=sr_1_1?__mk_pt_BR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&keywords=Me+Poupe%21%3A+10+passos+para+nunca+mais...&qid=1568583858&s=books&sr=1-1'

url_down_list = [r'/content/Do Mil ao Milhão. Sem Cortar o Cafezinho. - 9788595083271 - Livros na Amazon Brasil.html',
                  r'/content/O milagre da manhã - 9788576849940 - Livros na Amazon Brasil.html',
                  r'/content/A Sutil Arte de Ligar o F_da-Se_ Uma estratégia inusitada para uma vida melhor - 9788551002490 - Livros na Amazon Brasil.html',
                  r'/content/Mais esperto que o diabo_ O mistério revelado da liberdade e do sucesso - 9788568014004 - Livros na Amazon Brasil.html',
                  r'/content/Os segredos da mente milionária - 9788575422397 - Livros na Amazon Brasil.html',
                  r'/content/Pai rico, pai pobre 20 anos - 9788550801483 - Livros na Amazon Brasil.html',
                  r'/content/Bíblia King James Atualizada - 9788544105665 - Livros na Amazon Brasil.html',
                  r'/content/O poder da ação - 9788545200345 - Livros na Amazon Brasil.html',
                  r'/content/Como Fazer Amigos e Influenciar Pessoas - 9788504018028 - Livros na Amazon Brasil.html',
                  r'/content/Me Poupe!_ 10 passos para nunca mais faltar dinheiro no seu bolso - 9788543105819 - Livros na Amazon Brasil.html']



In [0]:
def amazon_single(site):
  # page = rq.get(url)
  # # print(page.text)

  # soup = bs(page.text,"html.parser")
  
  soup = bs(open(site),"html.parser")

  title = (soup.find('span', {'id':'productTitle'})).text
  
  author = (soup.find('span', {'class':'author notFaded'})).find('a',{'class':'a-link-normal'}).text
  
  book = soup.find('div', {'class':'content'}).find_all('li')[:5]
 
  n_pages = book[0].contents[1].strip().split(' ')[0]
  
  publishing = book[1].contents[1].strip().split(';')[0]
  
  language = book[2].contents[1].strip()
  
  cols = ['Título', 'Autor', 'Nº Páginas', 'Editora', 'Idioma']

  list_info = [title, author, n_pages, publishing, language]
  list_info
  book_pd = pd.DataFrame([list_info],\
                        columns=cols)
  
  return book_pd

In [0]:
for book in url_down_list:
  print(book,'\n')
  display(amazon_single(book))
  print('\n')

/content/Do Mil ao Milhão. Sem Cortar o Cafezinho. - 9788595083271 - Livros na Amazon Brasil.html 



,Título,Autor,Nº Páginas,Editora,Idioma
0,Do Mil ao Milhão. Sem Cortar o Cafezinho.,Thiago Nigro,192,HarperCollins,Português




/content/O milagre da manhã - 9788576849940 - Livros na Amazon Brasil.html 



,Título,Autor,Nº Páginas,Editora,Idioma
0,O milagre da manhã,Hal Elrod,196,BestSeller,Português




/content/A Sutil Arte de Ligar o F_da-Se_ Uma estratégia inusitada para uma vida melhor - 9788551002490 - Livros na Amazon Brasil.html 



...

,Título,Autor,Nº Páginas,Editora,Idioma
0,O poder da ação,Paulo Vieira,256,Gente,Português




/content/Como Fazer Amigos e Influenciar Pessoas - 9788504018028 - Livros na Amazon Brasil.html 



,Título,Autor,Nº Páginas,Editora,Idioma
0,Como Fazer Amigos e Influenciar Pessoas,Dale Carnegie,264,Companhia Editora Nacional,Português




/content/Me Poupe!_ 10 passos para nunca mais faltar dinheiro no seu bolso - 9788543105819 - Livros na Amazon Brasil.html 



,Título,Autor,Nº Páginas,Editora,Idioma
0,Me Poupe!: 10 passos para nunca mais faltar di...,Nathalia Arcuri,176,Editora Sextante,Português


##SUBMARINO <-- As requisições não funcionam

In [0]:
# url = 'https://www.submarino.com.br/produto/129142202/livro-cujo?pfm_carac=Stephen%20King&pfm_index=6&pfm_page=author&pfm_pos=grid&pfm_type=vit_product_grid&sellerId'
# url = 'https://www.submarino.com.br/produto/133133721?DCSext.recom=RR_item_page.rr1-ClickEV&nm_origem=rec_item_page.rr1-ClickEV&nm_ranking_rec=2'
url = 'https://www.submarino.com.br/autor/stephen-king?chave=prf_hi_at_lig_3_stephen-king'

In [0]:
page = rq.get(url)
  # print(page.text)

soup = bs(page.text,"html.parser")

## SARAIVA

In [0]:
sar_url_list = ['https://www.saraiva.com.br/it-a-coisa-7697879/p',
                'https://www.saraiva.com.br/a-barraca-do-beijo-edicao-economica-10601621/p',
                'https://www.saraiva.com.br/mulheres-cultura-e-politica-9727683/p',
                'https://www.saraiva.com.br/ma-feminista-ensaios-provocativos-de-uma-ativista-desastrosa-9342733/p',
                'https://www.saraiva.com.br/empodere-se-10182962/p',
                'https://www.saraiva.com.br/persepolis-completo-1999282/p',
                'https://www.saraiva.com.br/fome-uma-autobiografia-do-meu-corpo-9888989/p',
                'https://www.saraiva.com.br/no-seu-pescoco-9727573/p',
                'https://www.saraiva.com.br/mulheres-cultura-e-politica-9727683/p',
                'https://www.saraiva.com.br/sejamos-todos-feministas-8592230/p']

In [0]:
def saraiva_single(site):
  page = rq.get(site)

  soup = bs(page.text,"html.parser")

  title = soup.find('div', {'class':'product-review'}).h1.text

  infos = soup.find('table', {'class':'group Caracteristicas'})

  author = infos.find('td', {'class':'value-field Autor'}).text

  n_pages = infos.find('td', {'class':'value-field Numero-de-Paginas'}).text

  language = infos.find('td', {'class':'value-field Idioma'}).text

  binding = infos.find('td', {'class':'value-field Acabamento'}).text

  cols = ['Título', 'Autor', 'Nº Páginas', 'Acabamento', 'Idioma']

  list_info = [title, author, n_pages, binding, language]
  list_info
  book_pd = pd.DataFrame([list_info],\
                        columns=cols)

  return book_pd

In [52]:
for url in sar_url_list:
  print(url)
  display(saraiva_single(url))
  print('\n')

https://www.saraiva.com.br/it-a-coisa-7697879/p


,Título,Autor,Nº Páginas,Acabamento,Idioma
0,It - A Coisa,"King,Stephen",1104,Brochura,Português




https://www.saraiva.com.br/a-barraca-do-beijo-edicao-economica-10601621/p


,Título,Autor,Nº Páginas,Acabamento,Idioma
0,A Barraca do Beijo - Edição Econômica,"Reekles,Beeth",336,Brochura,Português




https://www.saraiva.com.br/mulheres-cultura-e-politica-9727683/p


...

,Título,Autor,Nº Páginas,Acabamento,Idioma
0,No Seu Pescoço,"Adichie,Chimamanda Ngozi",256,Brochura,Português




https://www.saraiva.com.br/mulheres-cultura-e-politica-9727683/p


,Título,Autor,Nº Páginas,Acabamento,Idioma
0,"Mulheres, Cultura e Política","Davis,Angela, Davis,Angela",200,Brochura,Português




https://www.saraiva.com.br/sejamos-todos-feministas-8592230/p


,Título,Autor,Nº Páginas,Acabamento,Idioma
0,Sejamos Todos Feministas,"Adichie,Chimamanda Ngozi",64,Brochura,Português
